In [1]:
import random
import csv
import os

In [2]:
# function to create a valid state for the tic tac toe game
# o represents player 1, x represents player 2, b represents blank
# the 3x3 grid has been appended along rows to give a 9 character string
def generate_state():
    num_x = random.randint(0, 4)  # Random number of 'x' (0 to 4)
    num_o = random.randint(num_x, num_x+1)  # Random number of 'o' (equal to or one more than 'x')
    num_b = 9 - num_x - num_o  # Remaining characters will be 'b'

    string_list = ['x'] * num_x + ['o'] * num_o + ['b'] * num_b
    random.shuffle(string_list)  # Shuffle the list to get random order
    return ''.join(string_list), num_o-num_x

In [3]:
#function to discard states in which the game has already ended
def check_positions(string):
    # Define positions to check for all 'x' or all 'o'
    positions_to_check = [
        [0, 1, 2],  # 1st row
        [3, 4, 5],  # 2nd row
        [6, 7, 8],  # 3rd row
        [0, 3, 6],  # 1st column
        [1, 4, 7],  # 2nd column
        [2, 5, 8],  # 3rd column
        [0, 4, 8],  # diagonal from top-left to bottom-right
        [2, 4, 6]   # diagonal from top-right to bottom-left
    ]
    
    for positions in positions_to_check:
        symbols = [string[pos] for pos in positions]
        if symbols.count('x') == 3 or symbols.count('o') == 3 or 'b' not in string:
            return True
    return False

In [4]:
# Test the function
random_string, player = generate_state()
print(random_string)
print(check_positions(random_string))
print(player)

bbbxbbbob
False
0


In [5]:
def check_two_in_a_row(grid_string):
    # Define the indices for all possible two-in-a-row combinations
    combinations = [
        [0, 1, 2],  # 1st row
        [3, 4, 5],  # 2nd row
        [6, 7, 8],  # 3rd row
        [0, 3, 6],  # 1st column
        [1, 4, 7],  # 2nd column
        [2, 5, 8],  # 3rd column
        [0, 4, 8],  # diagonal from top-left to bottom-right
        [2, 4, 6]   # diagonal from top-right to bottom-left
    ]

    for i, combination in enumerate(combinations):
        symbols = [grid_string[j] for j in combination]
        if (symbols.count('x') == 2 and symbols.count('o') == 0) or (symbols.count('o') == 2 and symbols.count('x') == 0):
            return combination  # Return the index of the combination (corresponding to row or column)
    return -1  # Return -1 if no such combination is found

In [6]:
def optimal_move(state, player):
    if player==0:
        piece = "o"
    else:
        piece = "x"
    combination = check_two_in_a_row(state)
    if combination == -1:
        if state[4] == "b":
            move = piece+"5"
            return move
        
        for i in [0,2,6,8]:
            if state[i] == 'b':
                move = piece+str(i+1)
                return move
        
        return piece+str(state.index('b')+1)
    else:
        for index in combination:
            if state[index] == 'b':
                return piece+str(index+1) 
        

In [7]:
random_string, player = generate_state()
optim = optimal_move(random_string, player)
print(random_string)
print(optim)

bobxbbbob
x5


In [8]:
def append_to_csv(filename, data):
    # Check if the file exists
    file_exists = os.path.isfile(filename)
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # If the file doesn't exist, write header
        if not file_exists:
            writer.writerow(['Game States', 'Optimal Moves'])  # Writing column headers
        writer.writerow(data)

In [9]:
i = 0
while i<10000:  
    state, player = generate_state()
    if check_positions(state):
        continue
    move = optimal_move(state, player)
    data = [state, move]
    append_to_csv('ttt_data_ppo_train.csv', data)
    i = i+1